In [ ]:
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, load_model
from keras.layers import *
from keras import backend as K
from keras import optimizers, callbacks
import numpy as np
import pandas as pd
import cv2, h5py

In [ ]:
batch_size = 64

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
        'data/',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
        'data/',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
base_model = Xception(weights='imagenet', include_top=False)

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(6, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)
# compile the model (should be done *after* setting layers to non-trainable)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [ ]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("xception-{val_loss:.3f}.new.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=0)

In [ ]:
print("Training Progress:")
model_log = model.fit_generator(train_generator, validation_data=validation_generator,
          epochs=100,
          callbacks=[tensorboard, model_checkpoints])

In [ ]:
for layer in base_model.layers[:46]:
    layer.trainable = False

for layer in base_model.layers[46:]:
    layer.trainable = True

#for layer in base_model.layers:
#    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
adadelta = optimizers.Adadelta(lr=0.5, rho=0.95, epsilon=None, decay=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [ ]:
print("Training Progress:")
model_log = model.fit_generator(train_generator, validation_data=validation_generator,
          epochs=100,
          callbacks=[tensorboard, model_checkpoints])